In [53]:
!pip install finnhub-python sqlalchemy vaderSentiment python-dotenv



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [54]:
import os
import finnhub
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from datetime import datetime

# Load environment variables
load_dotenv()
FINNHUB_API_KEY = os.getenv("FINNHUB_API_KEY")
DATABASE_URL = os.getenv("DATABASE_URL")

from models.sentiment_data_model import Base
Base.metadata.create_all(engine)

# Setup Finnhub client
finnhub_client = finnhub.Client(api_key=FINNHUB_API_KEY)

# Setup DB engine
engine = create_engine(DATABASE_URL)

# VADER setup
vader = SentimentIntensityAnalyzer()



In [55]:
load_dotenv()
FINNHUB_API_KEY = os.getenv("FINNHUB_API_KEY")
DATABASE_URL = os.getenv("DATABASE_URL")


In [56]:
def analyze_sentiment(text):
    score = vader.polarity_scores(text)['compound']
    label = 'POS' if score > 0.05 else 'NEG' if score < -0.05 else 'NEU'
    return score, label, 'VADER', None

def insert_sentiment(data_dict):
    with engine.begin() as conn:
        query = text("""
            INSERT INTO sentiment_data (
                ticker, source, api_vendor, date, headline, sentiment_score,
                sentiment_label, article_url, author, platform_tags,
                model_used, confidence, created_at
            ) VALUES (
                :ticker, :source, :api_vendor, :date, :headline, :sentiment_score,
                :sentiment_label, :article_url, :author, :platform_tags,
                :model_used, :confidence, :created_at
            )
        """)
        conn.execute(query, data_dict)


In [57]:
def fetch_finnhub_sentiment(ticker):
    try:
        news = finnhub_client.company_news(ticker, _from='2024-06-01', to=datetime.today().strftime('%Y-%m-%d'))
        for item in news:
            if not item.get('headline'):
                continue

            score, label, model, conf = analyze_sentiment(item['headline'])
            data = {
                'ticker': ticker,
                'source': item.get('source', 'Finnhub'),
                'api_vendor': 'Finnhub',
                'date': datetime.fromtimestamp(item['datetime']).date(),
                'headline': item['headline'],
                'sentiment_score': score,
                'sentiment_label': label,
                'article_url': item.get('url'),
                'author': item.get('source'),
                'platform_tags': 'Finnhub',
                'model_used': model,
                'confidence': conf,
                'created_at': datetime.utcnow().isoformat()
            }
            insert_sentiment(data)
        print(f"Inserted sentiment records for {ticker} from Finnhub")
    except Exception as e:
        print(f"Error fetching Finnhub data for {ticker}: {e}")


In [58]:
fetch_finnhub_sentiment("NVDA")


Inserted sentiment records for NVDA from Finnhub
